In [1]:
import numpy as np
import scipy as sp
import torchio as tio
import os, sys
import random
import matplotlib.pyplot as plt
random.seed(0)

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



In [2]:
root_dir = '/mnt/Data_ubuntu/datasets/CT_PET_Corr_Uncorr/'
list_subs = os.listdir(root_dir)
print(list_subs[0:3], len(list_subs))

with open('all_subs_mapping.txt', 'w') as f:
    for sid, l in enumerate(list_subs):
        f.write('{} -- {}\n'.format(l, sid))
        # id 0 to 30 training; 31 to 35 val; rest testing
        
def get_crop_according_To_ct(CT_img, PET_img, PET_corr_img, thr):
    mask = CT_img<thr
    _, X, Y, _ = np.where(mask==True)
    x1, y1 = X[0], Y[0]
    x2, y2 = X[-1], Y[-1]
    c1 = CT_img[0,x1:x2, y1:y2,:]
    c2 = PET_img[0,x1:x2, y1:y2,:]
    c3 = PET_corr_img[0,x1:x2, y1:y2,:]
    return c1, c2, c3
#     plt.imshow(CT_img[0,x1:x2, y1:y2,idx]/1000, cmap='gray')
#     plt.colorbar()
#     plt.show()
#     plt.imshow(PET_img[0,x1:x2, y1:y2,idx], cmap='gray')
#     plt.colorbar()
#     plt.show()
#     a = PET_img[0,x1:x2, y1:y2,idx]
#     plt.imshow(np.log(np.clip(a, a_min=1, a_max=np.max(a.numpy()))), cmap='gray')
#     plt.colorbar()
#     plt.show()

def get_ct_uncorr_corr_series(root_dir, subname):
    all_series = os.listdir(root_dir+subname)
    for s in all_series:
        if 'CT' in s:
            ct_img = tio.ScalarImage(root_dir+subname+'/'+s+'/')
        elif 'uncorr' in s:
            uncorr_img = tio.ScalarImage(root_dir+subname+'/'+s+'/')
        elif 'PET corr' in s:
            corr_img = tio.ScalarImage(root_dir+subname+'/'+s+'/')
    #transform all to uncorr
    transform = tio.Resample(uncorr_img)
    ct_img = transform(ct_img)
    corr_img = transform(corr_img)
    
    return ct_img.numpy(), uncorr_img.numpy(), corr_img.numpy()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/Data_ubuntu/datasets/CT_PET_Corr_Uncorr/'

In [41]:
for sid, s in enumerate(list_subs):
    ct_img, uncorr_img, corr_img = get_ct_uncorr_corr_series(root_dir, s)
    ct_crop, uncorr_crop, corr_crop = get_crop_according_To_ct(ct_img, uncorr_img, corr_img, 0)
    print(ct_crop.shape, uncorr_crop.shape, corr_crop.shape)
    #60 to 105 is roughly head region
    for i in range(60,ct_crop.shape[2]-10):
        from skimage.transform import resize
        ct_slice = resize(ct_crop[:,:,i], (196,196))
        print(np.max(ct_slice), np.min(ct_slice))
        uncorr_slice = resize(uncorr_crop[:,:,i], (196,196))
        print(np.max(uncorr_slice), np.min(uncorr_slice))
        corr_slice = resize(corr_crop[:,:,i], (196,196))
        print(np.max(corr_slice), np.min(corr_slice))
        
        np.save('./multimodal_slices/ct/sub{}_slice{}.npy'.format(sid, i), ct_slice)
        np.save('./multimodal_slices/uncorr/sub{}_slice{}.npy'.format(sid, i), uncorr_slice)
        np.save('./multimodal_slices/corr/sub{}_slice{}.npy'.format(sid, i), corr_slice)
        
#         print(i)
#         plt.figure(figsize=(12,5))
#         plt.subplot(1,3,1)
#         plt.imshow(ct_crop[:,:,i]/1000, cmap='gray')
#         plt.colorbar()
#         plt.subplot(1,3,2)
#         plt.imshow(np.log(np.clip(uncorr_crop[:,:,i], 1, np.max(uncorr_crop[:,:,i])))/10, cmap='gray')
#         plt.colorbar()
#         plt.subplot(1,3,3)
#         plt.imshow(np.log(np.clip(corr_crop[:,:,i], 1, np.max(corr_crop[:,:,i])))/10, cmap='gray')
#         plt.colorbar()
#         plt.show()
#     break

(170, 170, 116) (170, 170, 116) (170, 170, 116)
(145, 145, 116) (145, 145, 116) (145, 145, 116)
(175, 175, 116) (175, 175, 116) (175, 175, 116)
(144, 144, 116) (144, 144, 116) (144, 144, 116)
(175, 175, 116) (175, 175, 116) (175, 175, 116)
(140, 139, 116) (140, 139, 116) (140, 139, 116)
(133, 133, 116) (133, 133, 116) (133, 133, 116)
(141, 141, 116) (141, 141, 116) (141, 141, 116)
(135, 135, 116) (135, 135, 116) (135, 135, 116)
(137, 138, 116) (137, 138, 116) (137, 138, 116)
(152, 153, 116) (152, 153, 116) (152, 153, 116)
(130, 131, 116) (130, 131, 116) (130, 131, 116)
(160, 160, 116) (160, 160, 116) (160, 160, 116)
(148, 148, 116) (148, 148, 116) (148, 148, 116)
(135, 135, 116) (135, 135, 116) (135, 135, 116)
(154, 154, 116) (154, 154, 116) (154, 154, 116)
(130, 130, 116) (130, 130, 116) (130, 130, 116)
(154, 154, 116) (154, 154, 116) (154, 154, 116)
(137, 137, 116) (137, 137, 116) (137, 137, 116)
(160, 159, 116) (160, 159, 116) (160, 159, 116)
(152, 152, 116) (152, 152, 116) (152, 15

In [38]:
for sid, s in enumerate(list_subs):
    if sid<35:
        with open('train_split.txt', 'a') as f:
            for i in range(60,106):
                f.write('sub{}_slice{}.npy\n'.format(sid, i))
    elif sid>35 and sid<40:
        with open('val_split.txt', 'a') as f:
            for i in range(60,106):
                f.write('sub{}_slice{}.npy\n'.format(sid, i))
    elif sid>40:
        with open('test_split.txt', 'a') as f:
            for i in range(60,106):
                f.write('sub{}_slice{}.npy\n'.format(sid, i))